# Chapter 16. Operator Overloading

Main contents:
 - How an infix operator method should signal it cannot handle an operand
 - Using duck typing or goose typing to deal with operands of various types
 - special behaviour of the rich comparison operators (`==`, `>`, `>=`)
 - default handling of augmented assignment operators such as `+=` and how to overload them

## Operator Overloading 101

restrictions imposed by python
 - we cannot change the meaning of the operators for the built-in types
 - we cannot create new operators, only overload existing ones
 - a few operators can't be overloaded: `is`, `and`, `or`, `not` (but the bitwise can)


## Unary Operator

Note. Stick to the general rule of operators: always return a new object. i.e., don't modify the receiver (`self`), but create and return a new instance of a suitable type

In [1]:
# vector_v5.py
from array import array
import operator
import reprlib
import math
import functools
import itertools

class Vector:
  typecode = 'd'
  __match_args__ = ('x', 'y', 'z', 't')

  def __init__(self, components):
    # self._components - "protected" instance attribute will hold an array with the Vector components
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)
    components = components[components.find('['):-1]
    return f'Vector({components})'

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(self._components))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(*self)

  def __bool__(self):
    return bool(abs(self))

  def __len__(self):
    return len(self._components)

  def __getitem__(self, key):
    if isinstance(key, slice):
      cls = type(self)
      return cls(self._components[key])
    index = operator.index(key) # operator.index() calls the __index__ special method
    return self._components[index]

  def __getattr__(self, name):
    cls = type(self)
    try:
      pos = cls.__match_args__.index(name)
    except ValueError:
      pos = -1
    if 0 <= pos < len(self._components):
      return self._components[pos]
    msg = f'{cls.__name__!r} object has not attribute {name!r}'
    raise AttributeError(msg)

  def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1:
      if name in cls.__match_args__:
        error = 'readonly attribute {attr_name!r}'
      elif name.islower():
        error = "can't set attribute 'a' to 'z' in {cls_name!r}"
      else:
        error = ''
      if error:
        msg = error.format(cls_name=cls.__name__, attr_name=name)
        raise AttributeError(msg)
    super().__setattr__(name, value)

  def __eq__(self, other):
    return len(self) == len(other) and all(a == b for a, b in zip(self, other))

  def __hash__(self):
    hashes = map(hash, self._components)
    return functools.reduce(operator.xor, hashes, 0)

  def angle(self, n):
    r = math.hypot(*self[n:])
    a = math.atan2(r, self[n-1])
    if (n == len(self) - 1) and (self[-1] < 0):
      return math.pi * 2 - a
    else:
      return a

  def angles(self):
    return(self.angle(n) for n in range(1, len(self)))

  def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('h'):
      fmt_spec = fmt_spec[:-1]
      coords = itertools.chain([abs(self)], self.angles())
      outer_fmt = '<{}>'
    else:
      coords = self
      outer_fmt = '({})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(', '.join(components))

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

  def __neg__(self):
    return Vector(-x for x in self)

  def __pos__(self):
    return Vector(self)

### When x and +x are not equal

Everybody expects that `x == +x` but there are two cases in the standard library where `x != +x`

In [9]:
import decimal
ctx = decimal.getcontext()
ctx.prec = 40
one_third = decimal.Decimal('1') / decimal.Decimal('3')

In [10]:
one_third

Decimal('0.3333333333333333333333333333333333333333')

In [11]:
one_third == +one_third

False

In [12]:
ctx.prec = 28

In [13]:
one_third == +one_third

False

In [14]:
+one_third

Decimal('0.3333333333333333333333333333')

In [2]:
from collections import Counter

In [3]:
ct = Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [4]:
ct['r'] = -3
ct['d'] = 0

In [5]:
ct

Counter({'a': 5, 'b': 2, 'r': -3, 'c': 1, 'd': 0})

Note that `+ct` discards from the result any item with a negative or zero count.

In [6]:
+ct

Counter({'a': 5, 'b': 2, 'c': 1})

In [8]:
ct == +ct

False

## Overloading + for Vector Addition

In [16]:
# vector_v5.py
from array import array
import operator
import reprlib
import math
import functools
import itertools

class Vector:
  typecode = 'd'
  __match_args__ = ('x', 'y', 'z', 't')

  def __init__(self, components):
    # self._components - "protected" instance attribute will hold an array with the Vector components
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)
    components = components[components.find('['):-1]
    return f'Vector({components})'

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(self._components))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(*self)

  def __bool__(self):
    return bool(abs(self))

  def __len__(self):
    return len(self._components)

  def __getitem__(self, key):
    if isinstance(key, slice):
      cls = type(self)
      return cls(self._components[key])
    index = operator.index(key) # operator.index() calls the __index__ special method
    return self._components[index]

  def __getattr__(self, name):
    cls = type(self)
    try:
      pos = cls.__match_args__.index(name)
    except ValueError:
      pos = -1
    if 0 <= pos < len(self._components):
      return self._components[pos]
    msg = f'{cls.__name__!r} object has not attribute {name!r}'
    raise AttributeError(msg)

  def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1:
      if name in cls.__match_args__:
        error = 'readonly attribute {attr_name!r}'
      elif name.islower():
        error = "can't set attribute 'a' to 'z' in {cls_name!r}"
      else:
        error = ''
      if error:
        msg = error.format(cls_name=cls.__name__, attr_name=name)
        raise AttributeError(msg)
    super().__setattr__(name, value)

  def __eq__(self, other):
    return len(self) == len(other) and all(a == b for a, b in zip(self, other))

  def __hash__(self):
    hashes = map(hash, self._components)
    return functools.reduce(operator.xor, hashes, 0)

  def angle(self, n):
    r = math.hypot(*self[n:])
    a = math.atan2(r, self[n-1])
    if (n == len(self) - 1) and (self[-1] < 0):
      return math.pi * 2 - a
    else:
      return a

  def angles(self):
    return(self.angle(n) for n in range(1, len(self)))

  def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('h'):
      fmt_spec = fmt_spec[:-1]
      coords = itertools.chain([abs(self)], self.angles())
      outer_fmt = '<{}>'
    else:
      coords = self
      outer_fmt = '({})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(', '.join(components))

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

  def __neg__(self):
    return Vector(-x for x in self)

  def __pos__(self):
    return Vector(self)

  def __add__(self, other):
    # pairs is a generator that produces tuples (a, b)
    # where a is from self, and b is from other
    try:
      pairs = itertools.zip_longest(self, other, fillvalue=0)
      return Vector(a + b for a, b in pairs) #
    except TypeError:
      return NotImplemented

  def __radd__(self, other):
    return self + other

  # Equivalently,
  # __radd__ = __add__



In [17]:
v1 = Vector([3, 4, 5])
v1 + (10, 20, 30)

Vector([13.0, 24.0, 35.0])

In [18]:
(10, 20, 30) + Vector([3, 4, 5])

Vector([13.0, 24.0, 35.0])

In [20]:
v1 + "abc"

TypeError: unsupported operand type(s) for +: 'Vector' and 'str'

## Overloading * for Scalar Multiplication

Good Example of Goose Typing. If we tested `other` operand against `Vector`, we'd deny users the flexibility of using lists or arrays as operands to `@`

In [24]:
# vector_v5.py
from array import array
import operator
import reprlib
import math
import functools
import itertools

class Vector:
  typecode = 'd'
  __match_args__ = ('x', 'y', 'z', 't')

  def __init__(self, components):
    # self._components - "protected" instance attribute will hold an array with the Vector components
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)
    components = components[components.find('['):-1]
    return f'Vector({components})'

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(self._components))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(*self)

  def __bool__(self):
    return bool(abs(self))

  def __len__(self):
    return len(self._components)

  def __getitem__(self, key):
    if isinstance(key, slice):
      cls = type(self)
      return cls(self._components[key])
    index = operator.index(key) # operator.index() calls the __index__ special method
    return self._components[index]

  def __getattr__(self, name):
    cls = type(self)
    try:
      pos = cls.__match_args__.index(name)
    except ValueError:
      pos = -1
    if 0 <= pos < len(self._components):
      return self._components[pos]
    msg = f'{cls.__name__!r} object has not attribute {name!r}'
    raise AttributeError(msg)

  def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1:
      if name in cls.__match_args__:
        error = 'readonly attribute {attr_name!r}'
      elif name.islower():
        error = "can't set attribute 'a' to 'z' in {cls_name!r}"
      else:
        error = ''
      if error:
        msg = error.format(cls_name=cls.__name__, attr_name=name)
        raise AttributeError(msg)
    super().__setattr__(name, value)

  def __eq__(self, other):
    return len(self) == len(other) and all(a == b for a, b in zip(self, other))

  def __hash__(self):
    hashes = map(hash, self._components)
    return functools.reduce(operator.xor, hashes, 0)

  def angle(self, n):
    r = math.hypot(*self[n:])
    a = math.atan2(r, self[n-1])
    if (n == len(self) - 1) and (self[-1] < 0):
      return math.pi * 2 - a
    else:
      return a

  def angles(self):
    return(self.angle(n) for n in range(1, len(self)))

  def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('h'):
      fmt_spec = fmt_spec[:-1]
      coords = itertools.chain([abs(self)], self.angles())
      outer_fmt = '<{}>'
    else:
      coords = self
      outer_fmt = '({})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(', '.join(components))

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

  def __neg__(self):
    return Vector(-x for x in self)

  def __pos__(self):
    return Vector(self)

  def __add__(self, other):
    # pairs is a generator that produces tuples (a, b)
    # where a is from self, and b is from other
    try:
      pairs = itertools.zip_longest(self, other, fillvalue=0)
      return Vector(a + b for a, b in pairs) #
    except TypeError:
      return NotImplemented

  def __radd__(self, other):
    return self + other

  # Equivalently,
  # __radd__ = __add__

  def __mul__(self, scalar):
    try:
      scalar = float(scalar) # duck typing!
    except TypeError:
      return NotImplemented

    return Vector(scalar * val for val in self)

  def __rmul__(self, scalar):
    return self * scalar

In [25]:
v1 = Vector([1.0, 2.0, 3.0])
14 * v1

Vector([14.0, 28.0, 42.0])

In [26]:
v1 * True

Vector([1.0, 2.0, 3.0])

In [27]:
from fractions import Fraction
v1 * Fraction(1, 3)

Vector([0.3333333333333333, 0.6666666666666666, 1.0])

## Using @ as an Infix Operator

In [28]:
# vector_v7.py
from array import array
import operator
import reprlib
import math
import functools
import itertools
from collections import abc

class Vector:
  typecode = 'd'
  __match_args__ = ('x', 'y', 'z', 't')

  def __init__(self, components):
    # self._components - "protected" instance attribute will hold an array with the Vector components
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)
    components = components[components.find('['):-1]
    return f'Vector({components})'

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(self._components))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(*self)

  def __bool__(self):
    return bool(abs(self))

  def __len__(self):
    return len(self._components)

  def __getitem__(self, key):
    if isinstance(key, slice):
      cls = type(self)
      return cls(self._components[key])
    index = operator.index(key) # operator.index() calls the __index__ special method
    return self._components[index]

  def __getattr__(self, name):
    cls = type(self)
    try:
      pos = cls.__match_args__.index(name)
    except ValueError:
      pos = -1
    if 0 <= pos < len(self._components):
      return self._components[pos]
    msg = f'{cls.__name__!r} object has not attribute {name!r}'
    raise AttributeError(msg)

  def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1:
      if name in cls.__match_args__:
        error = 'readonly attribute {attr_name!r}'
      elif name.islower():
        error = "can't set attribute 'a' to 'z' in {cls_name!r}"
      else:
        error = ''
      if error:
        msg = error.format(cls_name=cls.__name__, attr_name=name)
        raise AttributeError(msg)
    super().__setattr__(name, value)

  def __eq__(self, other):
    return len(self) == len(other) and all(a == b for a, b in zip(self, other))

  def __hash__(self):
    hashes = map(hash, self._components)
    return functools.reduce(operator.xor, hashes, 0)

  def angle(self, n):
    r = math.hypot(*self[n:])
    a = math.atan2(r, self[n-1])
    if (n == len(self) - 1) and (self[-1] < 0):
      return math.pi * 2 - a
    else:
      return a

  def angles(self):
    return(self.angle(n) for n in range(1, len(self)))

  def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('h'):
      fmt_spec = fmt_spec[:-1]
      coords = itertools.chain([abs(self)], self.angles())
      outer_fmt = '<{}>'
    else:
      coords = self
      outer_fmt = '({})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(', '.join(components))

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

  def __neg__(self):
    return Vector(-x for x in self)

  def __pos__(self):
    return Vector(self)

  def __add__(self, other):
    # pairs is a generator that produces tuples (a, b)
    # where a is from self, and b is from other
    try:
      pairs = itertools.zip_longest(self, other, fillvalue=0)
      return Vector(a + b for a, b in pairs) #
    except TypeError:
      return NotImplemented

  def __radd__(self, other):
    return self + other

  # Equivalently,
  # __radd__ = __add__

  def __mul__(self, scalar):
    try:
      scalar = float(scalar) # duck typing!
    except TypeError:
      return NotImplemented

    return Vector(scalar * val for val in self)

  def __rmul__(self, scalar):
    return self * scalar

  def __matmul__(self, other):
    if (isinstance(other, abc.Sized) and
        isinstance(other, abc.Iterable)):
        if len(self) == len(other):
          return sum(a * b for a, b in zip(self, other))
        else:
          return ValueError("@ requires vectors of equal length")
    else:
      return NotImplemented

  def __rmatmul__(self, other):
    return self @ other

In [29]:
va = Vector([1, 2, 3])
vz = Vector([5, 6, 7])

va @ vz

38.0

## Rich Comparison Operators

Handling comparison operators by Python interpreter is similar to what we just saw, but differs in two important ways:
 - same set of methods is used in forward and reverse operator cllas.
 - in case of `==` and `!=`, if the reverse method is missing, or returns `NotImplemented`

In [30]:
va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1, 4))
va == vb

True

In [31]:
vc = Vector([1, 2])

In [32]:
t3 = (1, 2, 3)

In [33]:
va == t3

True

In [34]:
t3 == va

True

Taking a clue from Python, we fix the `__eq__` in our Vector class

In [35]:
[1, 2] == (1, 2)

False

In [36]:
# vector_v8.py
from array import array
import operator
import reprlib
import math
import functools
import itertools
from collections import abc

class Vector:
  typecode = 'd'
  __match_args__ = ('x', 'y', 'z', 't')

  def __init__(self, components):
    # self._components - "protected" instance attribute will hold an array with the Vector components
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)
    components = components[components.find('['):-1]
    return f'Vector({components})'

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(self._components))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(*self)

  def __bool__(self):
    return bool(abs(self))

  def __len__(self):
    return len(self._components)

  def __getitem__(self, key):
    if isinstance(key, slice):
      cls = type(self)
      return cls(self._components[key])
    index = operator.index(key) # operator.index() calls the __index__ special method
    return self._components[index]

  def __getattr__(self, name):
    cls = type(self)
    try:
      pos = cls.__match_args__.index(name)
    except ValueError:
      pos = -1
    if 0 <= pos < len(self._components):
      return self._components[pos]
    msg = f'{cls.__name__!r} object has not attribute {name!r}'
    raise AttributeError(msg)

  def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1:
      if name in cls.__match_args__:
        error = 'readonly attribute {attr_name!r}'
      elif name.islower():
        error = "can't set attribute 'a' to 'z' in {cls_name!r}"
      else:
        error = ''
      if error:
        msg = error.format(cls_name=cls.__name__, attr_name=name)
        raise AttributeError(msg)
    super().__setattr__(name, value)

  def __eq__(self, other):
    if isinstance(other, Vector):
      return len(self) == len(other) and all(a == b for a, b in zip(self, other))
    else:
      return NotImplemented

  def __hash__(self):
    hashes = map(hash, self._components)
    return functools.reduce(operator.xor, hashes, 0)

  def angle(self, n):
    r = math.hypot(*self[n:])
    a = math.atan2(r, self[n-1])
    if (n == len(self) - 1) and (self[-1] < 0):
      return math.pi * 2 - a
    else:
      return a

  def angles(self):
    return(self.angle(n) for n in range(1, len(self)))

  def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('h'):
      fmt_spec = fmt_spec[:-1]
      coords = itertools.chain([abs(self)], self.angles())
      outer_fmt = '<{}>'
    else:
      coords = self
      outer_fmt = '({})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(', '.join(components))

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

  def __neg__(self):
    return Vector(-x for x in self)

  def __pos__(self):
    return Vector(self)

  def __add__(self, other):
    # pairs is a generator that produces tuples (a, b)
    # where a is from self, and b is from other
    try:
      pairs = itertools.zip_longest(self, other, fillvalue=0)
      return Vector(a + b for a, b in pairs) #
    except TypeError:
      return NotImplemented

  def __radd__(self, other):
    return self + other

  # Equivalently,
  # __radd__ = __add__

  def __mul__(self, scalar):
    try:
      scalar = float(scalar) # duck typing!
    except TypeError:
      return NotImplemented

    return Vector(scalar * val for val in self)

  def __rmul__(self, scalar):
    return self * scalar

  def __matmul__(self, other):
    if (isinstance(other, abc.Sized) and
        isinstance(other, abc.Iterable)):
        if len(self) == len(other):
          return sum(a * b for a, b in zip(self, other))
        else:
          return ValueError("@ requires vectors of equal length")
    else:
      return NotImplemented

  def __rmatmul__(self, other):
    return self @ other

In [37]:
va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1, 4))
print(va == vb)

t3 = (1, 2, 3)
print(va == t3)

True
False


In [38]:
va != vb

False

In [39]:
vc != (1, 2, 3)

True

## Augmented Assignment Operators

Our `Vector` class already supports the augmented assignment operator `+=` and `*=`. That's because augmented assignment works with immutable receivers by creating new instances and rebinding the lefthand variable

In [41]:
v1 = Vector([1, 2, 3])
v1_alias = v1
id(v1)

135549427527840

In [42]:
v1 += Vector([4, 5, 6]) # new object is created
v1

Vector([5.0, 7.0, 9.0])

In [43]:
v1_alias

Vector([1.0, 2.0, 3.0])

In [44]:
id(v1)

135549427527312

In [45]:
v1 *= 11 # new object is created

In [46]:
v1

Vector([55.0, 77.0, 99.0])

In [47]:
id(v1)

135549427525008

## Augmented Assignment Operator

In [ ]:
# tombola.py

import abc

class Tombola(abc.ABC):

  @abc.abstractmethod
  def load(self, iterable):
    """Add items from an iterable"""

  @abc.abstractmethod
  def pick(self):
    """ Remove item at random, returning it

    This method should raise `LookupError` when the instance is empty
    """

  def loaded(self):
    return bool(self.inspect())

  def inspect(self):
    items = []
    while True:
      try:
        items.append(self.pick())
      except LookupError:
        break
    self.load(items)
    return tuple(items)



In [ ]:
# bingo.py
import random
from tombola import Tombola

class BingoCage(Tombola):
  def __init__(self, items):
    self._randomizer = random.SystemRandom()
    self._items = []
    self.load(items)

  def load(self, items):
    self._items.extend(items)
    self._randomizer.shuffle(self._items)

  def pick(self):
    try:
      return self._items.pop()
    except IndexError:
      raise LookupError('pick from empty BingoCage')

  def __call__(self):
    self.pick()

In [66]:
from tombola import Tombola
from bingo import BingoCage

class AddableBingoCage(BingoCage):
  def __add__(self, other):
    if isinstance(other, Tombola): # Our __add__ will only work with an instance of
                                   #   Tombola as the second operand
      return AddableBingoCage(self.inspect() + other.inspect())
    else:
      return NotImplemented

  def __iadd__(self, other):
    if isinstance(other, Tombola):
      other_iterable = other.inspect()
    else:
      try:
        other_iterable = iter(other)
      except TypeError:
        msg = ('right operand in += must be '
               "'Tombola' or an iterable")
        raise TypeError(msg)
    self.load(other_iterable)
    return self # very important: augmented assignment special methods must return self

In [67]:
# AddableBingoCage demo
vowels = "AEIOU"
globe = AddableBingoCage(vowels)
print(globe.inspect())
print(globe.pick() in vowels)
print(len(globe.inspect()) == 4)


('O', 'A', 'U', 'I', 'E')
True
True


In [68]:
globe2 = AddableBingoCage('XYZ')
globe3 = globe + globe2
assert len(globe3.inspect()) == 7

In [69]:
void = globe + [10, 20]

TypeError: unsupported operand type(s) for +: 'AddableBingoCage' and 'list'

In [72]:
globe_orig = globe
assert len(globe.inspect()) == 4

globe += globe2
assert len(globe.inspect()) == 7

globe += ['M', 'N']
assert len(globe.inspect()) == 9

assert globe is globe_orig

globe += 1

TypeError: right operand in += must be 'Tombola' or an iterable